In [ ]:
import re
import string
import pickle

In [ ]:
#  load lib sensitive
with open('./vietnam-sensitive-words/profanity_wordlist.txt', 'r', encoding="utf8") as f:
    list_bad_words = [word[:-1] for word in f]
with open('./vietnam-sensitive-words/block_words.txt', 'r', encoding="utf8") as f:
    block_words = [word[:-1] for word in f]
with open('./domain_shorteners/domain_shorteners.txt', 'r') as f:
    domain_shorteners = [domain[:-1] for domain in f]
with open('./scam-links/links.txt', 'r') as f:
    scam_links = [link[:-1] for link in f]
scam = scam_links + domain_shorteners
list_bad_words = list_bad_words + scam

In [ ]:
class CheckBadWords:
    def __init__(self, badwords, blockwords):
        self.pattern1 = '|'.join([r"\b" + badword + r"\b" for badword in badwords])
        self.pattern2 = '|'.join(blockwords)
    def __call__(self, str_):
        print(str_)
        if re.findall(self.pattern1, str_):
            return {"cleaned_text": str_, "Sensitive": 1, "Score": 0.99}
        elif re.findall(self.pattern2, str_):
            return {"cleaned_text": str_, "Sensitive": 1, "Score": 0.99}
        else:
            return {"cleaned_text": str_, "Sensitive": 0, "Score": 0}

In [ ]:
badwords_censor = CheckBadWords(list_bad_words, block_words)


In [ ]:
badwords_censor("đây là câu chuyện tb")

đây là câu chuyện tb


{'cleaned_text': 'đây là câu chuyện tb', 'Sensitive': 0, 'Score': 0}

In [ ]:
import torch

In [ ]:
# load stopwords vn
with open("vietnamese.txt", 'r', encoding="utf8") as f:
    stop_words = [word[:-1] for word in f]
len(stop_words)

1943

In [ ]:
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

c:\Users\ADMIN\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

In [ ]:
import torch.nn as nn

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
model = SentimentClassifier(n_classes=2)

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
model.to(device)
model.load_state_dict(torch.load('phobert_fold3.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
def infer(text, tokenizer, max_len=120):
    encoded_review = tokenizer.encode_plus(
        text,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    print(f'Text: {text}')
    print(f'Sentiment: ', [y_pred])

In [ ]:
infer('dm ngu đến mức đó thì chịu', tokenizer)

Text: dm ngu đến mức đó thì chịu
Sentiment:  [tensor([1])]
